# Aufgabe-C-Sturm

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/C_sturm herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen "ml_LaB_4$" anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Sagen Sie die Windgeschwindigkeit eines Sturms (in Knoten) aufgrund von Satellitenfotos vorher.
Der Datensatz besteht aus Fotos von 494 verschiedenen Stürmen im Atlantik und Pazifik mit ihren zugehörigen Windgeschwindigkeiten.
Jedes Bild hat `366 x 366` Pixel, und es sind 70.257 Trainingsdaten und 45.377 Testdaten vorhanden.
Die Bilder wurden zu mehreren Zeitpunkten während der Lebensdauer eines Sturms aufgenommen.

Für jeden Sturm im Trainings und Testdatensatz erhalten Sie eine Zeitreihe von Bildern mit der jeweiligen assoziierten relativen Zeit seit Beginn des Sturms.
Ihr Modell sollte neben den reinen Bilddaten also auch den zeitlichen Verlauf des Sturms betrachten, um die Vorhersage für künftige Vorhersagezeitpunkte zu erstellen

Die Bilder sind nach folgendem Schema benannt: `{image_id}.jpg`.
Diese IDs bestehen aus einer Sturm_ID und einer Bildnummer entsprechend der zeitlichen Bildreihenfolge.

Ihr Ziel ist es, für die Testdaten die korrekte Windgeschwindigkeiten vorherzusagen.

# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

Geplantes Vorgehen: 

- Daten vorbereitung
- CNN-Modell trainieren (für räumliche Merkmale der Bilder)
- LSTM-Modell trainieren (für zeitlichen Verlauf der Bilder)
- Kombinierung der Modelle (Fully Connected-Layer um CNN in LSTM einzuspeisen, Ausgabe von LSTM ist finale Vorhersage)
- Kombinierte Modell auf Trainingsdaten trainieren 

--------------------------------------------------------------------------------------------------------------------------------------------------------------

Das geplante Vorgehen für die Vorhersage von Windgeschwindigkeiten eines Sturms aufgrund von Satellitenfotos mithilfe von tiefen neuronalen Netzen kann wie folgt aussehen:

1. Datenanalyse: Zunächst sollten die Trainings- und Testdaten untersucht werden, um eine Vorstellung davon zu bekommen, wie die Daten strukturiert sind und wie die Windgeschwindigkeiten mit den Bilddaten und dem zeitlichen Verlauf des Sturms korrelieren.

2. Datenpräparation: Die Bilddaten können normalisiert und in eine geeignete Form gebracht werden, um sie für das Training des neuronalen Netzes zu verwenden. Dabei kann auch die Information über den zeitlichen Verlauf des Sturms in die Datensätze integriert werden.

3. Modellierung: Es kann ein tiefes neuronales Netzwerk entworfen werden, das die Bilddaten und die Information über den zeitlichen Verlauf des Sturms als Eingabe erhält und die Windgeschwindigkeit als Ausgabe generiert. Hierbei können verschiedene Architekturen von neuronalen Netzen ausprobiert werden, um das beste Ergebnis zu erzielen.

4. Training: Das entworfene Modell kann mit den Trainingsdaten trainiert werden. Dabei sollten geeignete Verlustfunktionen und Optimierer gewählt werden, um die Leistung des Modells zu verbessern.

5. Evaluierung: Nach dem Training sollte die Leistung des Modells mit den Testdaten ausgewertet werden, um zu überprüfen, ob es in der Lage ist, die Windgeschwindigkeiten korrekt vorherzusagen.

6. Feintuning: Wenn das Modell nicht zufriedenstellend abschneidet, kann es durch Feintuning verbessert werden, indem z.B. die Hyperparameter optimiert werden.

7. Anwendung: Wenn das Modell gut funktioniert, kann es für Vorhersagen von Windgeschwindigkeiten von Stürmen aufgrund von Satellitenfotos verwendet werden.

Es ist zu beachten, dass es bei jedem Schritt des oben beschriebenen Vorgehens viele Möglichkeiten und Entscheidungen gibt, die je nach den Eigenschaften der Daten und des Modells angepasst werden können.

In [24]:
int('000')

0

In [26]:
import os
import json
from PIL import Image

data_dir = 'data' # Verzeichnis in dem die Daten gespeichert sind

def load_data(data_type):
    data_path = os.path.join(data_dir, f'storm_{data_type}_data')
    label_path = os.path.join(data_dir,f'storm_{data_type}_labels')
    data = []
    labels = []

    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        label_file = os.path.join(label_path,folder_name, 'labels' + '.json')
        with open(label_file) as f:
            label_data = json.load(f)

        for file_name in os.listdir(folder_path):
            if not file_name.endswith('.jpg'):
                continue
            image_path = os.path.join(folder_path, file_name)
            image = Image.open(image_path)
            if image.mode != 'RGB':
                image = image.convert('RGB')
            data.append(image)

            # image_id = int(file_name[:-4])
            image_id = folder_name
            if image_id not in label_data:
                raise ValueError(f'Missing label data for image {file_name}')
            label = label_data[image_id]
            if not isinstance(label, (float, int)):
                raise ValueError(f'Invalid label for image {file_name}: {label}')
            labels.append(label)

    return data, labels

# Laden der Trainingsdaten
train_data, train_labels = load_data('train')
print(f'{len(train_data)} Trainingsbilder und {len(train_labels)} Trainingslabels gefunden.')

# Laden der Testdaten
test_data, test_labels = load_data('test')
print(f'{len(test_data)} Testbilder und {len(test_labels)} Testlabels gefunden.')

ValueError: Missing label data for image image.jpg

In [6]:
import os
import json
import numpy as np
from PIL import Image

data_dir = 'data' # Verzeichnis in dem die Daten gespeichert sind

def load_data(data_type):
    data_path = os.path.join(data_dir, f'storm_{data_type}_data')
    label_path = os.path.join(data_dir,f'storm_{data_type}_labels')
    data_arr = []
    label_arr = []

    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        for file_name in os.listdir(folder_path):
            if file_name == "features.json":
                data = [os.path.join(folder_path,file_name)]
            if not file_name.endswith(".jpg"):
                continue
            image_path = os.path.join(folder_path, file_name)
            with Image.open(image_path) as image:
                image = image.resize((366,366))
                image_array = np.array(image)
                data.append(image_array)
            id = folder_name
            data.append(id)
            data_arr.append(np.array(data))
    for folder_name in os.listdir(label_path):
        folder_path = os.path.join(label_path, folder_name)
        for file_name in os.listdir(folder_path):
            if not file_name == "labels.json":
                continue
            label_file = os.path.join(label_path,folder_name, 'labels' + '.json')
            with open(label_file) as l:
                label_data = json.load(l)
            label_arr.append(label_data)

    return np.array(data_arr), np.array(label_arr)


In [7]:
X_train, Y_train = load_data("train")

C:\Users\Lukas\AppData\Local\Temp\ipykernel_9056\498781141.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_arr.append(np.array(data))


In [8]:
X_test, Y_test = load_data("test")

C:\Users\Lukas\AppData\Local\Temp\ipykernel_9056\498781141.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_arr.append(np.array(data))


--------------------------------------------------------------------------------
Die Daten sind nun in die Arrays geladen. Aufgeteilt in trainings-daten und trainings-labels, sowie test-daten und test-labels.
Die Bilddateien werden auf eine einheitliche Größe skaliert und anschliessend werden die Pixel in einem Array normalisiert.